# LunarLander 🚀🌠
## Packages

In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import sys, os
os.chdir(r'C:\Users\Quentin\Documents\5 GMM Projet\save')  # to change the current directory

Using TensorFlow backend.


## Class Agent

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.96
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    # enregistre ce qu'il a fait et stock dans son deque(=liste avec limite)
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size) # Hazard
        # Approximation de la ligne de la Q table avec le réseau de neurone
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action ()
    
    def learn(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = (reward + self.gamma *
                      np.amax(self.model.predict(next_state)[0]))
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)

    # Rejoue et apprend batch_size événement au hazard dans son deque
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            self.learn(state, action, reward, next_state, done)
            
    def refresh(self):
        if self.epsilon > self.epsilon_min :
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


## Learning

In [4]:
env = gym.make('LunarLander-v2')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
#agent.load("cartpole-dqn.h5")
done = False
batch_size = 32
EPISODES = 100
print("State size : %d\nAction size : %d"%(state_size,action_size))

State size : 8
Action size : 4


In [5]:
for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size]) # Pour avour une matrice
    score = []
    for time in range(600):
        env.render()        
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        if reward > 60 : print("Atterissage")
        score.append(reward)
        #reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size]) # Pour avour une matrice
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {:.2}, e: {:.2}"
                  .format(e, EPISODES-1, np.mean(score), agent.epsilon))
            break
        if len(agent.memory) > batch_size: #On se souvient si on a joué au moins batch_size fois.
            agent.replay(batch_size)
    agent.refresh()
    if e % 10 == 8:
        agent.save("lunarlander-dqn.h5")
env.close()

episode: 0/99, score: -5.2, e: 1.0
episode: 1/99, score: -4.0, e: 0.96
episode: 2/99, score: -1.3, e: 0.92
episode: 3/99, score: -1.0, e: 0.88
episode: 4/99, score: -1.4, e: 0.85
episode: 5/99, score: -1.2, e: 0.82
episode: 6/99, score: -0.69, e: 0.78
episode: 7/99, score: -2.4, e: 0.75
episode: 8/99, score: -1.3, e: 0.72
episode: 9/99, score: -1.3, e: 0.69
episode: 10/99, score: -2.2, e: 0.66
episode: 11/99, score: -1.4, e: 0.64
episode: 12/99, score: -1.2, e: 0.61
episode: 13/99, score: -2.1, e: 0.59
episode: 14/99, score: -0.11, e: 0.56
episode: 15/99, score: -1.1, e: 0.54
episode: 16/99, score: -0.024, e: 0.52
episode: 17/99, score: -0.67, e: 0.5
episode: 18/99, score: -0.21, e: 0.48
episode: 19/99, score: -0.091, e: 0.46
episode: 21/99, score: -0.51, e: 0.42
episode: 23/99, score: -0.07, e: 0.39
episode: 24/99, score: -0.094, e: 0.38
episode: 26/99, score: -1.3, e: 0.35
episode: 27/99, score: -0.15, e: 0.33
episode: 28/99, score: -0.28, e: 0.32
episode: 31/99, score: -1.6, e: 0.28

KeyboardInterrupt: 

## Game with training

In [ ]:
total_test_episodes = 10
self.epsilon = 0.001

In [ ]:
for e in range(total_test_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size]) # Pour avour une matrice
    for time in range(500):
        env.render()        
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        #reward = reward if not done else -10
        state = next_state
        if done:
            print("episode: {}/{}, time: {}, e: {:.2}"
                  .format(e, EPISODES-1, time, agent.epsilon))
            break
env.close()